# SageMaker 모델 빌드 파이프라인을 이용한 모델 빌드 오케스트레이션
Amazon SageMaker Model building pipeline은 머신러닝 워크플로우를 개발하는 데이터 과학자, 엔지니어들에게 SageMaker작업과 재생산가능한 머신러닝 파이프라인을 오케스트레이션하는 기능을 제공합니다. 또한 커스텀빌드된 모델을 실시간 추론환경이나 배치변환을 통한 추론 실행환경으로 배포하거나, 생성된 아티팩트의 계보(lineage)를 추적하는 기능을 제공합니다. 이 기능들을 통해 모델 아티팩트를 배포하고, 업무환경에서의 워크플로우를 배포/모니터링하고, 간단한 인터페이스를 통해 아티팩트의 계보 추적하고, 머신러닝 애플리케이션 개발의 베스트 프렉티스를 도입하여, 보다 안정적인 머신러닝 애플리케이션 운영환경을 구현할 수 있습니다.

SageMaker pipeline 서비스는 JSON 선언으로 구현된 SageMaker Pipeline DSL(Domain Specific Language, 도메인종속언어)를 지원합니다. 이 DSL은 파이프라인 파라마터와 SageMaker 작업단계의 DAG(Directed Acyclic Graph)를 정의합니다. SageMaker Python SDK를 이용하면 이 파이프라인 DSL의 생성을 보다 간편하게 할 수 있습니다.

# 0. 사용 코드

### 0-1. preprocessing

In [1]:
%%writefile src/v2.0/preprocessing.py

import argparse
import os
import requests
import tempfile
import subprocess, sys

import pandas as pd
import numpy as np
from glob import glob
import copy
from collections import OrderedDict
from pathlib import Path
import joblib

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

import logging
import logging.handlers
from logging.config import dictConfig

import json
import base64
import boto3
from botocore.client import Config
from botocore.exceptions import ClientError

import time
from datetime import datetime as dt
import datetime
from pytz import timezone
from dateutil.relativedelta import *

###############################
######### 전역변수 설정 ##########
###############################
KST = dt.today() + relativedelta(hours=9)
KST_aday_before = KST - relativedelta(days=1) 
ric_list = ['BOc1', 'BOc2', 'BOc3','BOPLKL','BRRTSc1', 'BRRTSc2', 'BRRTSc3', 'CAD', 'EUR','JPY', 'KRW', 'MYR', 'GBP', 'INR','Cc1', 'Cc2', 'Cc3','CCMc1', 'CCMc2', 'CCMc3',
            'CLc1', 'CLc2', 'CLc3','CNY','COMc1', 'COMc2','COMc3','CTc1', 'CTc2', 'CTc3', 'DJCI', 'DJCIBR', 'DJCICL', 'DJCICN', 'DJCIEN', 'DJCIGR', 'DJCIIA', 'DJCING', 
            'DJCISO', 'DJCIWH', 'DJT','FCPOc1', 'FCPOc2', 'FCPOc3','FGVHKL',
            'GCc1', 'GCc2', 'GCc3','GENMKL','HSI','IOIBKL', 'KCc1', 'KCc2', 'KCc3','KLKKKL','KLSE','KQ11', 'KS11',
            'KWc1', 'KWc2', 'KWc3','LCOc1', 'LCOc2', 'LCOc3','LWBc1', 'LWBc2', 'LWBc3','MCCc1', 'MCCc2','MXSCKL','Oc1', 'Oc2', 'Oc3','PEPTKL','RRc1', 'RRc2', 'RRc3','RSc1', 'RSc2', 'RSc3',
            'Sc1', 'Sc2', 'Sc3','SIMEKL','SOPSKL', 'THPBKL', 'Wc1', 'Wc2', 'Wc3'
           ]
col_names_asis = ['ds','high','low','open','ric']
col_names_tobe = ['ds','high','low','open','y']

###############################
######### util 함수 설정 ##########
###############################
def _get_logger():
    '''
    로깅을 위해 파이썬 로거를 사용
    # https://stackoverflow.com/questions/17745914/python-logging-module-is-printing-lines-multiple-times
    '''
    loglevel = logging.DEBUG
    l = logging.getLogger(__name__)
    if not l.hasHandlers():
        l.setLevel(loglevel)
        l.addHandler(logging.StreamHandler(sys.stdout))        
        l.handler_set = True
    return l  
logger = _get_logger()

def download_object(file_name):
    try:
        s3_client = boto3.client("s3")
        download_path = Path('test') / file_name.replace('/','_')
        s3_client.download_file(
            BUCKET_NAME_USECASE,
            file_name,
            str(download_path)
        )
        return "Success"
    except Exception as e:
        return e

def download_parallel_multiprocessing(path_list):
    with ProcessPoolExecutor() as executor:
        future_to_key = {executor.submit(download_object, key): key for key in path_list}
        for future in futures.as_completed(future_to_key):
            key = future_to_key[future]
            exception = future.exception()
            if not exception:
                yield key, future.result()
            else:
                yield key, exception
                                
def get_list_in_s3(key_id : str,
                   secret_key_id : str,
                   bucket_name : str,
                   s3_path : str) -> list:
    
    s3 = boto3.client('s3',
                      aws_access_key_id = ACCESS_KEY_ID,
                      aws_secret_access_key = ACCESS_SECRET_KEY,
                      region_name = 'ap-northeast-2')
    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket = bucket_name,
                               Prefix = s3_path)  # 원하는 bucket 과 하위경로에 있는 object list # dict type
    contents_list = [] # object list의 Contents를 가져옴
    for page in pages:
        for obj in page['Contents']:
            contents_list.append(obj)
    return contents_list

def get_file_folders(s3_client, bucket_name, prefix=""):
    file_names = []
    folders = []

    default_kwargs = {
        "Bucket": bucket_name,
        "Prefix": prefix
    }
    next_token = ""

    while next_token is not None:
        updated_kwargs = default_kwargs.copy()
        if next_token != "":
            updated_kwargs["ContinuationToken"] = next_token

        response = s3_client.list_objects_v2(**default_kwargs)
        contents = response.get("Contents")

        for result in contents:
            key = result.get("Key")
            if key[-1] == "/":
                folders.append(key)
            else:
                file_names.append(key)

        next_token = response.get("NextContinuationToken")

    return file_names, folders


def download_files(s3_client, bucket_name, local_path, file_names, folders):

    local_path = Path(local_path)

    for folder in folders:
        folder_path = Path.joinpath(local_path, folder)
        folder_path.mkdir(parents=True, exist_ok=True)

    for file_name in file_names:
        file_path = Path.joinpath(local_path, file_name)
        file_path.parent.mkdir(parents=True, exist_ok=True)
        s3_client.download_file(
            bucket_name,
            file_name,
            str(file_path)
        )
        
def get_dataframe(base_preproc_input_dir, file_name_prefix ):    
    '''
    파일 이름이 들어가 있는 csv 파일을 모두 저장하여 데이터 프레임을 리턴
    '''
    
    input_files = glob('{}/{}*.csv'.format(base_preproc_input_dir, file_name_prefix))
    #claim_input_files = glob('{}/dataset*.csv'.format(base_preproc_input_dir))    
    logger.info(f"input_files: \n {input_files}")    
    
    if len(input_files) == 0:
        raise ValueError(('There are no files in {}.\n' +
                          'This usually indicates that the channel ({}) was incorrectly specified,\n' +
                          'the data specification in S3 was incorrectly specified or the role specified\n' +
                          'does not have permission to access the data.').format(base_preproc_input_dir, "train"))
        
    raw_data = [ pd.read_csv(file, index_col=0) for file in input_files ]
    df = pd.concat(raw_data)
   
    logger.info(f"dataframe shape \n {df.shape}")    
    logger.info(f"dataset sample \n {df.head(2)}")        
    #logger.info(f"df columns \n {df.columns}")    
    
    return df

def get_secret():

    secret_name = "dev/ForecastPalmOilPrice"
    region_name = "ap-northeast-2"
    
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name,
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException': # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException': # An error occurred on the server side.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException': # You provided an invalid value for a parameter.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException': # You provided a parameter value that is not valid for the current state of the resource.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException': # We can't find the resource that you asked for.
            raise e
    else:
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
            return secret
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            return decoded_binary_secret

        
def fill_missing_dates(df_in : pd.DataFrame,
                       freq : str
                      ) -> pd.DataFrame : 
    df = df_in.copy()
    if df["ds"].dtype == np.int64:
            df.loc[:, "ds"] = df.loc[:, "ds"].astype(str)
    df.loc[:, "ds"] = pd.to_datetime(df.loc[:, "ds"])
    r = pd.date_range(start = df["ds"].min(),
                      end = df["ds"].max(),
                      freq = freq)
    df = df.set_index("ds").reindex(r).rename_axis("ds").reset_index()
    return df

def fill_missing_price_value(df: pd.DataFrame, col: str, limit_linear : int = 20 ) -> pd.DataFrame :
    initial_is_na = sum(df[col].isnull())
    series = df.loc[:, col].astype(float)
    series = series.interpolate(method="linear", limit=limit_linear, limit_direction="both")
    series = [0 if v < 0 else v for v in series]
    df[col] = series
    #2023-01-07: Interpolate 했음에도 불구하고, null값인경우 close 값으로 채움
    df[col] = df[col].fillna(df['y'])
    return df

def scaling_value(df : pd.DataFrame,
                  col_name : str,
                  output_dir : str,
                  ric,) -> tuple:
                  # s3_resource,
                  # BUCKET_NAME_USECASE,
                  # S3_PATH_GOLDEN) -> tuple:

    series = df[col_name].values
    scaler = MinMaxScaler()
    series = series.reshape(-1,1)
    scaler.fit(series)
    series = scaler.transform(series)
    joblib.dump(scaler, os.path.join(output_dir, f'{ric}_{col_name}_scaler.pkl'))    
    return series

def convert_type(raw, cols, type_target):
    '''
    해당 데이터 타입으로 변경
    '''
    df = raw.copy()
    
    for col in cols:
        df[col] = df[col].astype(type_target)
    
    return df

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--base_preproc_input_dir', type=str, default="/opt/ml/processing/input") 
    parser.add_argument('--base_output_dir', type=str, default="/opt/ml/processing/output")
    parser.add_argument('--split_start', type=str, default='2014-07-02')    
    parser.add_argument('--split_end', type=str, default=KST.strftime('%Y-%m-%d'))
    parser.add_argument('--num_fold', type=str, default='5')
    parser.add_argument('--label_column', type=str, default="ric") 
    parser.add_argument("--scaler_switch", type = str, default = '1', help = '1이면 Scaling ON, 0이면 Scaling OFF')
     
    return parser.parse_args()

if __name__=='__main__':
    ################################
    ###### 커맨드 인자 파싱   ##########
    ################################
    
    logger.info("######### Argument Info ####################################")

    logger.info("### Argument Info ###")
    args = parse_args()
    logger.info(f"args.base_preproc_input_dir: {args.base_preproc_input_dir}")    
    logger.info(f"args.base_output_dir: {args.base_output_dir}")
    logger.info(f"args.label_column: {args.label_column}")        
    logger.info(f"args.split_start: {args.split_start}")   
    logger.info(f"args.split_end: {args.split_end}")   
    logger.info(f"args.scaler_switch: {args.scaler_switch}")
    logger.info(f"args.num_fold: {args.num_fold}")
    
    base_output_dir = args.base_output_dir
    base_preproc_input_dir = args.base_preproc_input_dir
    label_column = args.label_column
    split_start = args.split_start
    split_end = args.split_end
    num_fold = int(args.num_fold)
    scaler_switch = int(args.scaler_switch)
    
    ############################################
    ###### Secret Manager에서 키값 가져오기  #######
    ########################################### 
    logger.info(f"### Loading the key value using Secret Manager")

    keychain = json.loads(get_secret())
    ACCESS_KEY_ID = keychain['AWS_ACCESS_KEY_ID']
    ACCESS_SECRET_KEY = keychain['AWS_ACCESS_SECRET_KEY']

    BUCKET_NAME_USECASE = keychain['PROJECT_BUCKET_NAME']
    DATALAKE_BUCKET_NAME = keychain['DATALAKE_BUCKET_NAME']

    S3_PATH_STAGE = keychain['S3_PATH_STAGE']
    S3_PATH_GOLDEN = keychain['S3_PATH_GOLDEN']
    S3_PATH_TRAIN = keychain['S3_PATH_TRAIN']

    boto_session = boto3.Session(ACCESS_KEY_ID, ACCESS_SECRET_KEY)
    region = boto_session.region_name
    s3_resource = boto_session.resource('s3')
    s3_client = boto_session.client('s3')
    ############################################
    ###### 1. 데이터 Integration  #######
    ########################################### 
    total_start = time.time()
    start = time.time()
    logger.info(f"### Data Integration")
    path_list = []
    df_sum = pd.DataFrame()

    for (path, dir, files) in os.walk(base_preproc_input_dir):
        for filename in files:
            ext = os.path.splitext(filename)[-1]
            if ext == '.csv':
                path_list.append("%s/%s" % (path, filename))
                
    logger.info(f"The number for data : {len(path_list)}")
    equalSignList = ['CAD=', 'EUR=', 'JPY=', 'KRW=', 'MYR=', 'GBP=', 'INR=', 'CNY=']
    for file in path_list:
        try:
            df_tmp = pd.read_csv(file, encoding='utf-8') 
            df_tmp['RIC'] = df_tmp['RIC'].apply(lambda x: x[:-1] if x in equalSignList else x)
            df_sum = pd.concat([df_sum, df_tmp])
        except:
            logger.info(f"{file} is empty ")
    df_sum = df_sum.sort_values(by='Date').reset_index(drop=True)
    df_sum.to_csv(f"{base_output_dir}/stage/stage_integrated.csv", index = False)
    end = time.time()
    
    logger.info(f"Data Integration is done")
    logger.info(f"Runtime : {end - start:.1f} sec({((end - start)/60):.1f} min)")
    logger.info(f"The number for data : {len(path_list)}")
    logger.info(f"Integrated data sample: head(2) \n {df_sum.head(2)}")
    logger.info(f"Integrated data sample: tail(2) \n {df_sum.tail(2)}")
    logger.info(f"Integrated data.info \n {df_sum.info()}")

    #################################
    ####   2. 첫번쨰 전처리 단계     ####
    ####   품목선별, 열 삭제, 형변환  ####
    ################################    
    start = time.time()
    logger.info(f"\n ### RIC Item selection")    
    df_sum = df_sum[df_sum['RIC'].isin(ric_list)].reset_index()
    logger.info(f"The number for data after RIC Item selection : {df_sum.shape}")

    logger.info(f"\n ### Column selection")    
    
    df_sum = df_sum[['Date','HIGH', 'LOW', 'OPEN', 'CLOSE','RIC']]
    logger.info(f"The number for data after Column selection : {df_sum.shape}")
    logger.info(f"\n ### type conversion")    
    df_sum.loc[:, "Date"] = pd.to_datetime(df_sum.loc[:, "Date"])
    df_sum.loc[:, "HIGH"] = df_sum.loc[:, "HIGH"].astype(np.float32)
    df_sum.loc[:, "LOW"] = df_sum.loc[:, "LOW"].astype(np.float32)
    df_sum.loc[:, "OPEN"] = df_sum.loc[:, "OPEN"].astype(np.float32)
    df_sum.loc[:, "CLOSE"] = df_sum.loc[:, "CLOSE"].astype(np.float32)
    
    ####################################################
    ####   3. Autogluon timeseries 데이터 셋으로 만들기  ####
    ####################################################
    logger.info(f"\n ### Autogluon timeseriesdataframe Conversion")        
    df_list = OrderedDict()
    for name in ric_list:
        df_tmp = df_sum[df_sum['RIC'] == name]
        df_tmp = df_tmp.drop('RIC', axis=1)
        df_list[name] = df_tmp[df_tmp['Date'] >= split_start].reset_index(drop = True)
        
    ####################################################
    ############   4. 열 이름 변경, 결측치 처리  ############
    ###################################################
    logger.info(f"\n ### Rename columns")        
    col_names = ['ds','high','low','open','y']
    for name, value in df_list.items():
        df_list[name].columns = col_names

    logger.info(f"\n ### Fill missing value (Date)")        
    for name, value in df_list.items():
        df_list[name]  = fill_missing_dates(value, 'B')
        num_added = len(df_list[name]) - len(value)
        is_na = sum(df_list[name]['y'].isnull())
    
    logger.info(f"\n ### Fill missing value (Price)")        
    for name, value in df_list.items():
        df_proc1 = fill_missing_price_value(value, 'y')
        df_proc1 = fill_missing_price_value(value, 'high')
        df_proc1 = fill_missing_price_value(value, 'low')
        df_proc1 = fill_missing_price_value(value, 'open')
        df_list[name] = df_proc1
        
    ####################################################
    #################   5. Scaling  ###################
    ###################################################
    if scaler_switch == 1:
        logger.info(f"\n ### Scaling")            
        scale_dir = f"{base_output_dir}/scaler-files"
        os.makedirs(scale_dir, exist_ok=True)
        for name, value in df_list.items():
            for col in ['y','high','open','low']:
                value.loc[:, col] = scaling_value(value,
                                                  col,
                                                  f'{base_output_dir}/scaler',
                                                  name,) #s3_client, BUCKET_NAME_USECASE, S3_PATH_GOLDEN)
            df_list[name] = value
    else:
        logger.info(f"\n ### No Scaling")
    end = time.time()
    logger.info(f"\n### All Date Transform is done")
    print(f"All Date Transform Run time : {end - start:.1f} sec({((end - start)/60):.1f} min)")
    logger.info(f"The number for data : {len(path_list)}")
    logger.info(f"Transform data info \n {df_sum.head(2)}")
    #################################################
    #####   6. 훈련, 테스트 데이터 세트로 분리 및 저장  ######
    #################################################
    logger.info(f"\n ### Split train, test dataset")            
    df_golden = pd.DataFrame()
    for name, value in df_list.items():
        value = value.assign(ric = name)
        df_golden = pd.concat([df_golden, value])
        
    df_golden = df_golden.reset_index(drop = True)
    df_train_fold0 = df_golden[df_golden['ds'] < split_end]

    # train 데이터 나누기
    for cnt in range(num_fold):
        split_end = (dt.strptime(split_end, '%Y-%m-%d') - relativedelta(days=30)).strftime('%Y-%m-%d')
    
        logger.info(f"df_train_fold{cnt+1} = df_train_fold{cnt}[df_train_fold{cnt}['ds'] < {split_end}]")
        exec(f"df_train_fold{cnt+1} = df_train_fold{cnt}[df_train_fold{cnt}['ds'] < split_end]")
        exec(f"df_train_fold{cnt+1}.to_csv('{base_output_dir}/train/train_fold{cnt+1}.csv', index = False)")

        logger.info(f"df_test_fold{cnt+1} = df_train_fold{cnt}[df_train_fold{cnt}['ds'] >= {split_end}]")
        exec(f"df_test_fold{cnt+1} = df_train_fold{cnt}[df_train_fold{cnt}['ds'] >= split_end]")
        exec(f"df_test_fold{cnt+1}.to_csv('{base_output_dir}/test/test_fold{cnt+1}.csv', index = False)")
    
    logger.info(f"\n### End All of data preprocessing")
    
    total_end = time.time()
    print(f"Run time 시간 : {total_end - total_start:.1f} sec({((total_end - total_start)/60):.1f} min)\n")

Overwriting src/v2.0/preprocessing.py


### 0-2. train 

In [2]:
%%writefile src/v2.0/train.py

import argparse
import os
import requests
import tempfile
import subprocess, sys
import json

import glob
import pandas as pd
import joblib # from sklearn.externals import joblib
import pickle
import tarfile # model registry에는 uri만 등록된다.
from io import StringIO, BytesIO

import logging
import logging.handlers
from logging.config import dictConfig

from dateutil.relativedelta import *
from datetime import datetime as dt
import time

import boto3

KST = dt.today() + relativedelta(hours=9)

###############################
######### util 함수 설정 ##########
###############################
def _get_logger():
    '''
    로깅을 위해 파이썬 로거를 사용
    # https://stackoverflow.com/questions/17745914/python-logging-module-is-printing-lines-multiple-times
    '''
    loglevel = logging.DEBUG
    l = logging.getLogger(__name__)
    if not l.hasHandlers():
        l.setLevel(loglevel)
        logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))        
        l.handler_set = True
    return l  
logger = _get_logger()

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--train_dir", type=str, default='/opt/ml/processing/input/train')
    parser.add_argument("--test_dir", type=str, default='/opt/ml/processing/input/test')
    parser.add_argument('--output_dir', type = str, default = '/opt/ml/processing/output')
    parser.add_argument('--item', type = str, default = 'FCPOc3')
    parser.add_argument('--target', type = str, default = 'y')
    parser.add_argument('--metric', type = str, default = 'MAPE')    
    parser.add_argument('--quality', type = str, default = 'fast_training')    
    return parser.parse_args()

def create_tarfile(source_dir, output_filename=None):
    ''' create a tarfile from a source directory'''
    if output_filename == None:
        output_filename = "%s/tmptar.tar" %(tempfile.mkdtemp())
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))
    return output_filename 

def make_tarfile(source_dir, output_filename):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))
    return os.path.join(source_dir, output_filename)

if __name__ == "__main__":
    ############################################
    ########## 필요 라이브러리 설치  ###########
    ########################################### 
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'autogluon==0.6.1'])
    from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
    
    ######################################
    ## 커맨드 인자, Hyperparameters 처리 ##
    ######################################
    logger.info("######### Argument Info ####################################")
    logger.info("### start training code")    
    logger.info("### Argument Info ###")
    args = parse_args()
        
    logger.info(f"args.train_dir: {args.train_dir}")   
    logger.info(f"args.test_dir: {args.test_dir}")   
    logger.info(f"args.output_dir: {args.output_dir}")    
    logger.info(f"args.item: {args.item}")   
    logger.info(f"args.target: {args.target}")    
    logger.info(f"args.metric: {args.metric}")   
    logger.info(f"args.quality: {args.quality}")   
    
    train_dir = args.train_dir
    test_dir = args.test_dir
    output_dir = args.output_dir
    prediction_dir = os.path.join(output_dir, 'prediction')
    leaderboard_dir = os.path.join(output_dir, 'leaderboard')
    model_dir = os.path.join(output_dir, 'model')
    
    for path in [prediction_dir, leaderboard_dir, model_dir]:
        if not os.path.exists(path):
            os.mkdir(path)
    item = args.item
    target = args.target
    metric = args.metric
    quality = args.quality
    
    trlist = sorted(os.listdir(train_dir))
    telist = sorted(os.listdir(test_dir))
    
    logger.info(f"the list of train data {trlist}")
    logger.info(f"the list of train data {telist}")
    
    for train_file, test_file in zip(trlist, telist):
        logger.info("### Reading input data")
        logger.info(f"### train data: {train_file}")
        logger.info(f"### test data: {test_file}")
        
        df_train = pd.read_csv(os.path.join(train_dir, train_file))
        df_test = pd.read_csv(os.path.join(test_dir, test_file))      
        # df_train = df_train[df_train['ric'] != 'MCCc3']
        # df_test = df_test[df_test['ric'] != 'MCCc3']
        
        logger.info("### Convert TimeSeriesDataFrame")
        df_train.loc[:, "ds"] = pd.to_datetime(df_train.loc[:, "ds"])
        df_test.loc[:, "ds"] = pd.to_datetime(df_test.loc[:, "ds"])

        tdf_train = TimeSeriesDataFrame.from_data_frame(
            df_train,
            id_column="ric",
            timestamp_column="ds",
        )
        tdf_test = TimeSeriesDataFrame.from_data_frame(
            df_test,
            id_column="ric",
            timestamp_column="ds",
        )

        logger.info("### Show the range of date for training and test")    
        logger.info('Item:', item)
        logger.info('Target:', target)   
        logger.info('Train:',tdf_train.loc[item][target].index.min(),'~',tdf_train.loc[item][target].index.max())
        logger.info('Test:',tdf_test.loc[item][target].index.min(),'~',tdf_test.loc[item][target].index.max())
        logger.info('The number of test data:',len(tdf_test.loc[item][target]))

        logger.info("### Training AutoGluon Model")    
        predictor = TimeSeriesPredictor(
            path = model_dir,
            target = target,
            prediction_length = len(tdf_test.loc[item][target]),
            eval_metric = metric,
        )
        predictor.fit(
            train_data = tdf_train,
            presets = quality
        )
        logger.info("the list of data in model_dir {}".format(os.listdir(model_dir)))
        tar_file_path = make_tarfile(model_dir, f'{model_dir}/model.tar.gz')
        logger.info("Saving model to {}".format(tar_file_path))

        predictor_leaderboard = predictor.leaderboard(tdf_test, silent = True)
        predictor_leaderboard = predictor_leaderboard.sort_values(by = ['score_val', 'score_test'],
                                                                  ascending = False)
        predictor_leaderboard.to_csv(os.path.join(leaderboard_dir,
                                                  f'leaderboard-{test_file}'),
                                     index = False)
        logger.info(f"predictor_leaderboard sample: head(2) \n {predictor_leaderboard.head(2)}")
        
        top_model_name = predictor_leaderboard.loc[0, 'model']
        # second_model_name = predictor_leaderboard.loc[1, 'model']
        
        prediction_ag_model_01 = predictor.predict(data = tdf_train,
                                                   model = top_model_name)
#         prediction_ag_model_02 = predictor.predict(data = tdf_train,
#                                                    model = second_model_name)
        pred_result_01 = pd.merge(tdf_test.loc['FCPOc3']['y'], prediction_ag_model_01.loc['FCPOc3'],
                                  left_index = True, right_index = True, how = 'left')
        pred_result_01.to_csv(os.path.join(prediction_dir,
                                           f'pred-{top_model_name}-{test_file}'))   
        # pred_result_02 = pd.merge(tdf_test.loc['FCPOc3']['y'], prediction_ag_model_02.loc['FCPOc3'],
        #                           left_index = True, right_index = True, how = 'left')
        # pred_result_02.to_csv(os.path.join(prediction_dir,
        #                                    f'pred-{second_model_name}-{test_file}'))   

Overwriting src/v2.0/train.py


### 0-3. validation

In [3]:
%%writefile src/v2.0/model_validation.py

import glob
import os
import pandas as pd

from collections import defaultdict
import numpy as np
from collections import Counter

import time
from datetime import datetime as dt
import argparse
import json
import boto3
from io import StringIO, BytesIO
import joblib
import sys
import subprocess
import logging
import logging.handlers
import calendar
import tarfile


###############################
######### util 함수 설정 ##########
###############################
def _get_logger():
    loglevel = logging.DEBUG
    l = logging.getLogger(__name__)
    if not l.hasHandlers():
        l.setLevel(loglevel)
        logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))        
        l.handler_set = True
    return l  
logger = _get_logger()

def get_secret():
    secret_name = "dev/ForecastPalmOilPrice"
    region_name = "ap-northeast-2"
    
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name,
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException': # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException': # An error occurred on the server side.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException': # You provided an invalid value for a parameter.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException': # You provided a parameter value that is not valid for the current state of the resource.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException': # We can't find the resource that you asked for.
            raise e
    else:
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
            return secret
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            return decoded_binary_secret

def check_performance_threshold(iput_df : pd.DataFrame,
                                identifier: str,
                                threshold : float = -100):
    tmp = {}
    satisfied_df = iput_df[iput_df['score_val'] > threshold]
    if len(satisfied_df) > 0:
        tmp['identifier'] = identifier
        tmp['model'] = list(satisfied_df['model'])
        tmp['performance'] = list(satisfied_df['score_val'])
    return tmp

def get_model_performance_report(data):
    result = defaultdict(list)
    models_ext = [row["model"] for row in data if row]
    models = [item for sublist in models_ext for item in sublist]
    performance_ext = [row["performance"] for row in data if row]
    performance = [item for sublist in performance_ext for item in sublist]
    
    count_models = Counter(models)
    
    for keys, values in zip(models, performance):
        result[keys].append(values)

    for key, values in result.items():
        result[key] = []
        result[key].append(count_models[key])
        result[key].append(sum(values) / len(values))
        result[key].append(np.std(values))
    
    # 정렬 1순위 : 비즈니스담당자의 Metric에 선정된 Count 높은 순, 2순위: 표준편차가 작은 순(음수처리)
    result = sorted(result.items(), key=lambda k_v: (k_v[1][0], -k_v[1][2]), reverse=True) 
    return result

def register_model_in_aws_registry(model_zip_path: str,
                                   model_package_group_name: str,
                                   model_description: str,
                                   model_tags: dict,############################# parameter 추가할것: golden train path와 test path 
                                   model_status: str,
                                   sm_client) -> str:
    create_model_package_input_dict = {
        "ModelPackageGroupName": model_package_group_name,
        "ModelPackageDescription": model_description, # ex AutoGluon - WeightedEnsemble
        "CustomerMetadataProperties": model_tags,
        "ModelApprovalStatus": model_status,
        "InferenceSpecification": {
            "Containers": [
                {
                    "Image": '763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/autogluon-inference:0.4-cpu-py38',
                    "ModelDataUrl": model_zip_path
                }
            ],
            "SupportedContentTypes": ["text/csv"],
            "SupportedResponseMIMETypes": ["text/csv"],
        }
    }
    create_model_package_response = sm_client.create_model_package(**create_model_package_input_dict)
    model_package_arn = create_model_package_response["ModelPackageArn"]
    return model_package_arn


def register_manifest(source_path,
                      target_path,
                      s3_client,
                      BUCKET_NAME_USECASE):
    template_json = {"fileLocations": [{"URIPrefixes": []}],
                     "globalUploadSettings": {
                         "format": "CSV",
                         "delimiter": ","
                     }}
    paginator = s3_client.get_paginator('list_objects_v2')
    response_iterator = paginator.paginate(Bucket = BUCKET_NAME_USECASE,
                                           Prefix = source_path.split(BUCKET_NAME_USECASE+'/')[1]
                                          )
    for page in response_iterator:
        for content in page['Contents']:
            template_json['fileLocations'][0]['URIPrefixes'].append(f's3://{BUCKET_NAME_USECASE}/'+content['Key'])
    with open(f'./manifest_testing.manifest', 'w') as f:
        json.dump(template_json, f, indent=2)

    res = s3_client.upload_file('./manifest_testing.manifest',
                                BUCKET_NAME_USECASE,
                                f"{target_path.split(BUCKET_NAME_USECASE+'/')[1]}/visual_validation.manifest")
    return f"{target_path.split(BUCKET_NAME_USECASE+'/')[1]}/visual_validation.manifest"
    
def refresh_of_spice_datasets(user_account_id,
                              qs_data_name,
                              manifest_file_path,
                              BUCKET_NAME_USECASE,
                              qs_client):
    
    ds_list = qs_client.list_data_sources(AwsAccountId='108594546720')
    datasource_ids = [summary["DataSourceId"] for summary in ds_list["DataSources"] if qs_data_name in summary["Name"]]    
    for datasource_id in datasource_ids:
        response = qs_client.update_data_source(
            AwsAccountId=user_account_id,
            DataSourceId=datasource_id,
            Name=qs_data_name,
            DataSourceParameters={
                'S3Parameters': {
                    'ManifestFileLocation': {
                        'Bucket': BUCKET_NAME_USECASE,
                        'Key':  manifest_file_path
                    },
                },
            })
        logger.info(f"datasource_id:{datasource_id} 의 manifest를 업데이트: {response}")
    
    res = qs_client.list_data_sets(AwsAccountId = user_account_id)
    datasets_ids = [summary["DataSetId"] for summary in res["DataSetSummaries"] if qs_data_name in summary["Name"]]
    ingestion_ids = []

    for dataset_id in datasets_ids:
        try:
            ingestion_id = str(calendar.timegm(time.gmtime()))
            qs_client.create_ingestion(DataSetId = dataset_id,
                                       IngestionId = ingestion_id,
                                       AwsAccountId = user_account_id)
            ingestion_ids.append(ingestion_id)
        except Exception as e:
            logger.info(e)
            pass
    for ingestion_id, dataset_id in zip(ingestion_ids, datasets_ids):
        while True:
            response = qs_client.describe_ingestion(DataSetId = dataset_id,
                                                    IngestionId = ingestion_id,
                                                    AwsAccountId = user_account_id)
            if response['Ingestion']['IngestionStatus'] in ('INITIALIZED', 'QUEUED', 'RUNNING'):
                time.sleep(5)     #change sleep time according to your dataset size
            elif response['Ingestion']['IngestionStatus'] == 'COMPLETED':
                print("refresh completed. RowsIngested {0}, RowsDropped {1}, IngestionTimeInSeconds {2}, IngestionSizeInBytes {3}".format(
                    response['Ingestion']['RowInfo']['RowsIngested'],
                    response['Ingestion']['RowInfo']['RowsDropped'],
                    response['Ingestion']['IngestionTimeInSeconds'],
                    response['Ingestion']['IngestionSizeInBytes']))
                break
            else:
                logger.info("refresh failed for {0}! - status {1}".format(dataset_id,
                                                                          response['Ingestion']['IngestionStatus']))
                break
    return response

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--leaderboard_path', type=str, default="/opt/ml/processing/input/leaderboard")   
    parser.add_argument('--algorithm_name', type=str)
    parser.add_argument('--model_base_path', type=str)
    parser.add_argument('--manifest_base_path', type=str)
    parser.add_argument('--prediction_base_path', type=str)
    parser.add_argument('--threshold', type=str, default="-100")   
    parser.add_argument('--model_package_group_name', type=str, default = BUCKET_NAME_USECASE)  
    parser.add_argument('--qs_data_name', type=str, default = 'model_result')    

    return parser.parse_args()


if __name__=='__main__':
    logger.info(f"\n### Loading Key value from Secret Manager")
    keychain = json.loads(get_secret())
    ACCESS_KEY_ID = keychain['AWS_ACCESS_KEY_ID']
    ACCESS_SECRET_KEY = keychain['AWS_ACCESS_SECRET_KEY']
    BUCKET_NAME_USECASE = keychain['PROJECT_BUCKET_NAME']
    DATALAKE_BUCKET_NAME = keychain['DATALAKE_BUCKET_NAME']
    S3_PATH_REUTER = keychain['S3_PATH_REUTER']
    S3_PATH_WWO = keychain['S3_PATH_WWO']
    S3_PATH_STAGE = keychain['S3_PATH_STAGE']
    S3_PATH_GOLDEN = keychain['S3_PATH_GOLDEN']
    S3_PATH_TRAIN = keychain['S3_PATH_TRAIN']
    S3_PATH_FORECAST = keychain['S3_PATH_PREDICTION']
    
    boto3_session = boto3.Session(aws_access_key_id = ACCESS_KEY_ID,
                                  aws_secret_access_key = ACCESS_SECRET_KEY,
                                  region_name = 'ap-northeast-2')
    
    s3_client = boto3_session.client('s3')
    sm_client = boto3_session.client('sagemaker')
    qs_client = boto3_session.client('quicksight')

    sts_client = boto3_session.client("sts")
    user_account_id = sts_client.get_caller_identity()["Account"]
    ######################################
    ## 커맨드 인자, Hyperparameters 처리 ##
    ######################################
    args = parse_args()
    logger.info("######### Argument Info ####################################")
    logger.info("### start training code")    
    logger.info("### Argument Info ###")
    logger.info(f"args.algorithm_name: {args.algorithm_name}")    
    logger.info(f"args.leaderboard_path: {args.leaderboard_path}")    
    logger.info(f"args.model_base_path: {args.model_base_path}")
    logger.info(f"args.manifest_base_path: {args.manifest_base_path}")
    logger.info(f"args.prediction_base_path: {args.prediction_base_path}")
    logger.info(f"args.threshold: {args.threshold}")
    logger.info(f"args.model_package_group_name: {args.model_package_group_name}")
    logger.info(f"args.qs_data_name: {args.qs_data_name}")
  
    algorithm_name = args.algorithm_name
    leaderboard_path = args.leaderboard_path
    model_base_path = args.model_base_path
    manifest_base_path = args.manifest_base_path
    prediction_base_path = args.prediction_base_path
    threshold = float(args.threshold)
    model_package_group_name = args.model_package_group_name
    qs_data_name = args.qs_data_name
    
    lb_list = sorted(os.listdir(leaderboard_path))
    logger.info(f"leaderboard file list in {leaderboard_path}: {lb_list}")
    satisfied_info = []
    train_data_base_path = manifest_base_path
    test_data_base_path = manifest_base_path
    train_replace_dict = {'trained-model' : S3_PATH_GOLDEN,
                          'manifest' : 'train'}
    for key in train_replace_dict.keys():
        train_data_base_path = train_data_base_path.replace(key, train_replace_dict[key])
    test_replace_dict = {'trained-model' : S3_PATH_GOLDEN,
                          'manifest' : 'test'}
    for key in test_replace_dict.keys():
        test_data_base_path = test_data_base_path.replace(key, test_replace_dict[key])
        
    for idx, f_path in enumerate(lb_list):
        leaderboard = pd.read_csv(f'{leaderboard_path}/{f_path}').sort_values(by = ['score_val', 'score_test'],
                                                                              ascending = False)
        satisfied_info.append(check_performance_threshold(iput_df = leaderboard,
                                                          identifier = f'fold{idx}',
                                                          threshold = threshold))
    model_report = get_model_performance_report(satisfied_info)
    logger.info(f"\n####model_report: {model_report}")
    if model_report[0][1][0] == len(lb_list): # Fold 내 모든 성능이 비즈니스 담당자가 설정한 값을 만족한다면
        logger.info(f"\n#### Pass the 1st minimum performance valiation")
        manifest_file_path = register_manifest(prediction_base_path, 
                                               manifest_base_path,
                                               s3_client,
                                               BUCKET_NAME_USECASE)

        model_package_arn = register_model_in_aws_registry(model_zip_path = f"{model_base_path}/model.tar.gz",
                                                           model_package_group_name = model_package_group_name,
                                                           model_description = algorithm_name,
                                                           model_tags = {'champion_model' : str(model_report[0][0]),
                                                                         'passed_the_number_of_folds' : str(model_report[0][1][0]),
                                                                         'average_metric' : str(model_report[0][1][1]),
                                                                         'std_metric' : str(model_report[0][1][2]),
                                                                         'train_data' : str(train_data_base_path),
                                                                         'test_data' : str(test_data_base_path),
                                                                        },
                                                           model_status = 'PendingManualApproval',
                                                           sm_client = sm_client)
        logger.info('### Passed ModelPackage Version ARN : {}'.format(model_package_arn))
        logger.info('### manifest_file_path : {}'.format(manifest_file_path))
        res = refresh_of_spice_datasets(user_account_id,
                                        qs_data_name,
                                        manifest_file_path,
                                        BUCKET_NAME_USECASE,
                                        qs_client)
        logger.info('### refresh_of_spice_datasets : {}'.format(res))
    else:
        logger.info(f"\n#### Filtered at 1st valiation")
        model_package_arn = register_model_in_aws_registry(model_zip_path = f"{model_base_path}/model.tar.gz",
                                                           model_package_group_name = model_package_group_name,
                                                           model_description = algorithm_name,
                                                           model_tags = {'champion_model' : str(model_report[0][0]),
                                                                         'passed_the_number_of_folds' : str(model_report[0][1][0]),
                                                                         'average_metric' : str(model_report[0][1][1]),
                                                                         'std_metric' : str(model_report[0][1][2]),
                                                                         'train_data' : str(train_data_base_path),
                                                                         'test_data' : str(test_data_base_path),
                                                                        },
                                                           model_status = 'Rejected',
                                                           sm_client = sm_client)
        logger.info('### Rejected ModelPackage Version ARN : {}'.format(model_package_arn))

Overwriting src/v2.0/model_validation.py


# 1. 환경설정


## 1.1 라이브러리 및 변수 로딩

In [4]:
import argparse
import os
import requests
import tempfile
import subprocess, sys

import pandas as pd
import numpy as np
from glob import glob
import copy
from collections import OrderedDict
from pathlib import Path
import joblib

import logging
import logging.handlers

import json
import base64
import boto3
import sagemaker
from botocore.client import Config
from botocore.exceptions import ClientError

import time
from datetime import datetime as dt
import calendar
import datetime
from pytz import timezone
from dateutil.relativedelta import *

from sagemaker.mxnet import MXNet
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker import get_execution_role
from sagemaker.image_uris import retrieve
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.inputs import TrainingInput

In [5]:
# 한국 시간
KST = dt.today() + relativedelta(hours=9)
KST_aday_before = KST - relativedelta(days=1) 
yyyy, mm, dd = str(KST_aday_before.year), str(KST_aday_before.month).zfill(2), str(KST_aday_before.day).zfill(2)
print(f"Start job time: {KST}")

Start job time: 2023-03-24 16:37:01.391408


In [6]:
# 코드 버전
code_version = '2.0'
print(f"Code version: {code_version}")

Code version: 2.0


In [7]:
def get_secret():
    secret_name = "dev/ForecastPalmOilPrice"
    region_name = "ap-northeast-2"
    
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name,
    )
    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException': # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException': # An error occurred on the server side.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException': # You provided an invalid value for a parameter.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException': # You provided a parameter value that is not valid for the current state of the resource.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException': # We can't find the resource that you asked for.
            raise e
    else:
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
            return secret
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            return decoded_binary_secret

keychain = json.loads(get_secret())
ACCESS_KEY_ID = keychain['AWS_ACCESS_KEY_ID']
ACCESS_SECRET_KEY = keychain['AWS_ACCESS_SECRET_KEY']
BUCKET_NAME_USECASE = keychain['PROJECT_BUCKET_NAME']
DATALAKE_BUCKET_NAME = keychain['DATALAKE_BUCKET_NAME']
S3_PATH_REUTER = keychain['S3_PATH_REUTER']
S3_PATH_WWO = keychain['S3_PATH_WWO']
S3_PATH_STAGE = keychain['S3_PATH_STAGE']
S3_PATH_GOLDEN = keychain['S3_PATH_GOLDEN']
S3_PATH_TRAIN = keychain['S3_PATH_TRAIN']
S3_PATH_FORECAST = keychain['S3_PATH_PREDICTION']

boto3_session = boto3.Session(aws_access_key_id = ACCESS_KEY_ID,
                              aws_secret_access_key = ACCESS_SECRET_KEY,
                              region_name = 'ap-northeast-2')

s3_client = boto3_session.client('s3')
sm_client = boto3_session.client('sagemaker')
qs_client = boto3_session.client('quicksight')

sts_client = boto3_session.client("sts")
user_account_id = sts_client.get_caller_identity()["Account"]

노트북에 저장된 변수를 확인

In [8]:
!aws s3 cp 'src/v2.0' 's3://crude-palm-oil-prices-forecast/src' --recursive --exclude ".ipynb_checkpoints*"

upload: src/v2.0/preprocessing.py to s3://crude-palm-oil-prices-forecast/src/preprocessing.py
upload: src/v2.0/prediction-autogluon.py to s3://crude-palm-oil-prices-forecast/src/prediction-autogluon.py
upload: src/v2.0/model_validation.py to s3://crude-palm-oil-prices-forecast/src/model_validation.py
upload: src/v2.0/train.py to s3://crude-palm-oil-prices-forecast/src/train.py
upload: src/v2.0/visualization.py to s3://crude-palm-oil-prices-forecast/src/visualization.py


In [9]:
preprocessing_code = 's3://crude-palm-oil-prices-forecast/src/preprocessing.py'
training_code = 's3://crude-palm-oil-prices-forecast/src/train.py'
model_validation_code = 's3://crude-palm-oil-prices-forecast/src/model_validation.py'
print('code_version:',code_version)
print('preprocessing_code:',preprocessing_code)
print('training_code:',training_code)
print('model_validation_code:',model_validation_code)

code_version: 2.0
preprocessing_code: s3://crude-palm-oil-prices-forecast/src/preprocessing.py
training_code: s3://crude-palm-oil-prices-forecast/src/train.py
model_validation_code: s3://crude-palm-oil-prices-forecast/src/model_validation.py


# 2. 모델 빌딩 파이프라인 의 스텝(Step) 생성

## 2.1 모델 빌딩 파이프라인 변수 생성

In [10]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)
###################
# 0) 변수 선언 ###
##################
# 한국 시간
KST = dt.today() + relativedelta(hours=9)
KST_aday_before = KST - relativedelta(days=1) 
yyyy, mm, dd = str(KST_aday_before.year), str(KST_aday_before.month).zfill(2), str(KST_aday_before.day).zfill(2)

split_start = '2018-01-01'
split_end = KST.strftime('%Y-%m-%d')
timestamp = time.mktime(KST.timetuple())

# 프로젝트 변수
project_prefix = BUCKET_NAME_USECASE
usecase_base_path = os.path.join('s3://', BUCKET_NAME_USECASE)
datalake_base_path = os.path.join('s3://', DATALAKE_BUCKET_NAME, BUCKET_NAME_USECASE)
raw_data_path = os.path.join(datalake_base_path, 'EikonDataAPI')

# S3 디렉토리 위치(data path)
staged_data_dir = os.path.join(usecase_base_path, keychain['S3_PATH_STAGE'], KST_aday_before.strftime('%Y/%m/%d'),str(timestamp))
golden_data_dir = os.path.join(usecase_base_path, keychain['S3_PATH_GOLDEN'], KST_aday_before.strftime('%Y/%m/%d'),str(timestamp))
trained_model_dir = os.path.join(usecase_base_path, keychain['S3_PATH_TRAIN'], KST_aday_before.strftime('%Y/%m/%d'),str(timestamp))

# 전처리 결과 데이터 위치(Golden data path)
train_data_dir = os.path.join(golden_data_dir,'train')
test_data_dir = os.path.join(golden_data_dir,'test')
scaler_data_dir = os.path.join(golden_data_dir,'scaler-files')

num_fold = '3'
scaler_switch = '1' # '0': scaling, '1': scaling
print(f"Start job time: {KST}")
print(f"code verison: {code_version}")

###################
## 1) 데이터 전처리를 위한 파이프라인 변수  ####################################
###################
processing_instance_type = ParameterString(
    name = "ProcessingInstanceType",
    default_value = "ml.c5.xlarge" # cpu 성능이 더 중요하기 때문에 m5보다 비교적 가격이 저렴한 c5.xlarge를 선택하였다.
)
processing_instance_count = ParameterInteger(
    name = "ProcessingInstanceCount",
    default_value = 1
)
###################
## 2) 데이터 학습을 위한 파이프라인 변수  ####################################
###################
train_instance_type = ParameterString(
    name = "TrainingInstanceType",
    default_value = "ml.m5.xlarge"
)
train_instance_count = ParameterInteger(
    name = "TrainInstanceCount",
    default_value = 1
)
###################
## 3) 모델 검증을 위한 파이프라인 변수  ####################################
###################
model_validation_instance_count = ParameterInteger(
    name="ModelValidationInstanceCount",
    default_value=1
)
model_validation_instance_type = ParameterString(
    name="ModelValidationInstanceType",
    default_value='ml.c5.xlarge'
)

Start job time: 2023-03-24 16:37:02.816433
code verison: 2.0


## 2.2 프로세서 단계 정의

In [12]:
role = sagemaker.get_execution_role()
skframework_version = "1.0-1"#"0.23-1"
item = 'FCPOc3'
target = 'y'
metric = 'MAPE'
quality = 'medium_quality'#'medium_quality'#'fast_training'
###################
# 1) 데이터 전처리 ###
##################
skprocessor_preprocessing = SKLearnProcessor(
    framework_version = skframework_version,
    instance_type = "ml.c5.xlarge",
    instance_count = 1,
    base_job_name = f"{BUCKET_NAME_USECASE}(Preprocessing)",
    role = role,
)

################
# 2) 모델 학습 ###
###############
image_uri = retrieve(framework='mxnet',
                     region='ap-northeast-2',
                     version='1.9.0',
                     py_version='py38',
                     image_scope='training',
                     instance_type="ml.m5.xlarge")

script_processor_training = ScriptProcessor(
    command=['python3'],
    image_uri=image_uri,
    instance_type = "ml.m5.xlarge",
    instance_count = 1,
    base_job_name = f"{BUCKET_NAME_USECASE}(Train Model)",
    role = role,
)
################
# 3) 모델 검증 ###
###############
skprocessor_model_validation = SKLearnProcessor(
    framework_version = skframework_version,
    instance_type = "ml.c5.xlarge",
    instance_count = 1,
    base_job_name = f"{BUCKET_NAME_USECASE}(Model Validation)",
    role = role,
)

## 2.3 파이프라인 스텝 단계 정의

In [13]:
###################
# 1) 데이터 전처리 ###
##################
step_preprocessing = ProcessingStep(
    name = f"{BUCKET_NAME_USECASE}-Preprocessing",
    processor = skprocessor_preprocessing,
    inputs = [
        ProcessingInput(input_name = 'input_stage_data_path',
                        source = raw_data_path,
                        destination = '/opt/ml/processing/input'),
    ],
    outputs = [
        ProcessingOutput(output_name = "stage",
                         source = '/opt/ml/processing/output/stage',
                         destination = staged_data_dir),
        ProcessingOutput(output_name = "scaler",
                         source = '/opt/ml/processing/output/scaler',
                         destination = scaler_data_dir),        
        ProcessingOutput(output_name = "train",
                         source = '/opt/ml/processing/output/train',
                         destination = train_data_dir),
        ProcessingOutput(output_name = "test",
                         source = '/opt/ml/processing/output/test',
                         destination = test_data_dir),
    ],
    job_arguments = ["--split_start", split_start,
                     "--split_end", split_end,
                     "--num_fold", num_fold,
                     "--scaler_switch", scaler_switch], 
    code = preprocessing_code,
)
################
# 2) 모델 학습 ###
###############

step_train = ProcessingStep(
    name = f"{BUCKET_NAME_USECASE}-Training",
    processor = script_processor_training,
    inputs=[
        ProcessingInput(source = step_preprocessing.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
                        destination = "/opt/ml/processing/input/train"),
        ProcessingInput(source = step_preprocessing.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
                        destination = "/opt/ml/processing/input/test"),
    ],
    outputs=[
        ProcessingOutput(output_name = "prediction_data",
                         source = "/opt/ml/processing/output/prediction",
                         destination = f'{trained_model_dir}/prediction'),
        ProcessingOutput(output_name = "leaderboard_data",
                         source = "/opt/ml/processing/output/leaderboard",
                         destination = f'{trained_model_dir}/leaderboard'),        
        ProcessingOutput(output_name = "model_data",
                         source = "/opt/ml/processing/output/model",
                         destination = f'{trained_model_dir}/model'),        
        ProcessingOutput(output_name = "manifest_data",
                         source = "/opt/ml/processing/output/manifest",
                         destination = f'{trained_model_dir}/manifest')
        ],
    job_arguments = ["--item", item,
                     "--target", target,
                     "--metric", metric,
                     "--quality", quality],
    code = training_code
)

################
# 3) 모델 검증 ###
###############
# 참조:https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html#API_DescribeTrainingJob_ResponseSyntax
step_model_validaion = ProcessingStep(
    name = f"{project_prefix}-Model_validation",
    processor = skprocessor_model_validation,
    inputs=[
        ProcessingInput(source = step_train.properties.ProcessingOutputConfig.Outputs["leaderboard_data"].S3Output.S3Uri,
                        destination = "/opt/ml/processing/input/leaderboard")
    ],
    job_arguments = ["--algorithm_name", 'Autogluon',
                     "--model_base_path", step_train.properties.ProcessingOutputConfig.Outputs["model_data"].S3Output.S3Uri,
                     "--manifest_base_path", step_train.properties.ProcessingOutputConfig.Outputs["manifest_data"].S3Output.S3Uri,
                     "--prediction_base_path", step_train.properties.ProcessingOutputConfig.Outputs["prediction_data"].S3Output.S3Uri,
                     "--threshold", "-100",
                     "--model_package_group_name", BUCKET_NAME_USECASE,
                     "--qs_data_name", "model_result",
              ],
    code = model_validation_code
)

# 3. 최종 파이프라인 정의 및 실행

### 3.1 최종 파이프라인 정의
1. Processing
   1. 데이터 추출: 적재된 DataLake에서 Use-Case에 맞는 Stage data를 추출 → S3(staged data 폴더) 적재
   2. 데이터 정제: 추출된 Stage data를 불러옴 → 머신러닝(AI/ML) 입력값에 맞게 정제된 golden data로 변환 → S3(golden data폴더) 적재
2. Training
   1. 정제된 golden data를 불러옴 → 머신러닝(AI/ML) 학습 → 모델 파일과 모델 결과를 적재
3. Model Validation
   1. 1차 모델 검증 →  Model Registry 저장.
   2. 비즈니스 담당자 검증을 위한 시각화 data source 업데이트.   
   
참조: [MLOps Pipeline](https://aws.amazon.com/ko/blogs/machine-learning/deploy-an-mlops-solution-that-hosts-your-model-endpoints-in-aws-lambda/)

In [14]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        # 1) preprocessing's parameters 
        processing_instance_type, 
        processing_instance_count,
        # 2) training's parameters        
        train_instance_type,        
        train_instance_count,   
        # 3) model validating's parameters
        model_validation_instance_type,
        model_validation_instance_count,
    ],
   steps=[step_preprocessing,
          step_train,
          step_model_validaion]
)

### 3.2 파이프라인 정의 확인

In [15]:
import json

definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ProcessingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.c5.xlarge'},
  {'Name': 'ProcessingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'TrainInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'ModelValidationInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.c5.xlarge'},
  {'Name': 'ModelValidationInstanceCount',
   'Type': 'Integer',
   'DefaultValue': 1}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'crude-palm-oil-prices-forecast-Preprocessing',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': 'ml.c5.xlarge',
      'InstanceCount': 1,
      'VolumeSizeInGB': 30}},
    'AppSpecification': {'ImageUri': '3

### 3.3 파이프라인 정의를 제출하고 실행하기

In [16]:
%%time
start = time.time()
pipeline.upsert(role_arn = sagemaker.get_execution_role())
execution = pipeline.start()
#실행이 완료될 때까지 기다린다.
execution.wait() 
end = time.time()

WaiterError: Waiter PipelineExecutionComplete failed: Max attempts exceeded

In [18]:
print(f"데이터 전처리~모델 검증시간 : {end - start:.1f} sec")
print(f"데이터 전처리~모델 검증시간 : {((end - start)/60):.1f} min")

NameError: name 'end' is not defined

[23년 3월 1일 테스트]    
- 데이터 전처리 시간(초) = 20m22s
- 모델링 시간(초) = 12m54s
- 모델 검증 시간(초) = 4m22s
- 총계: 37m42s

### 3.4 Debug Model Validation 

In [20]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:108594546720:pipeline/crude-palm-oil-prices-forecast',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-northeast-2:108594546720:pipeline/crude-palm-oil-prices-forecast/execution/o0i36853z9qt',
 'PipelineExecutionDisplayName': 'execution-1679643425098',
 'PipelineExecutionStatus': 'Succeeded',
 'PipelineExperimentConfig': {'ExperimentName': 'crude-palm-oil-prices-forecast',
  'TrialName': 'o0i36853z9qt'},
 'CreationTime': datetime.datetime(2023, 3, 24, 7, 37, 5, 28000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2023, 3, 24, 8, 32, 30, 877000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': 'f3b18869-08b2-4d62-bc9a-890347be3de7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f3b18869-08b2-4d62-bc9a-890347be3de7',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '541',
   'date': 'Fri, 24 Mar 2023 08:35:28 GMT'},
  'RetryAttempts': 0}}

In [21]:
sm_client.list_pipelines()

{'PipelineSummaries': [{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:108594546720:pipeline/crude-palm-oil-prices-forecast',
   'PipelineName': 'crude-palm-oil-prices-forecast',
   'PipelineDisplayName': 'crude-palm-oil-prices-forecast',
   'RoleArn': 'arn:aws:iam::108594546720:role/service-role/AmazonSageMaker-ExecutionRole-20220901T154875',
   'CreationTime': datetime.datetime(2023, 1, 5, 23, 2, 43, 874000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2023, 3, 24, 8, 32, 30, 881000, tzinfo=tzlocal())}],
 'ResponseMetadata': {'RequestId': '870f4846-cd87-4229-9137-fe31d82dbfba',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '870f4846-cd87-4229-9137-fe31d82dbfba',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '400',
   'date': 'Fri, 24 Mar 2023 08:35:29 GMT'},
  'RetryAttempts': 0}}

In [22]:
execution.list_steps()

[{'StepName': 'crude-palm-oil-prices-forecast-Model_validation',
  'StartTime': datetime.datetime(2023, 3, 24, 8, 28, 2, 772000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2023, 3, 24, 8, 32, 30, 352000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:108594546720:processing-job/pipelines-o0i36853z9qt-crude-palm-oil-price-vcuzd43ncw'}}},
 {'StepName': 'crude-palm-oil-prices-forecast-Training',
  'StartTime': datetime.datetime(2023, 3, 24, 7, 57, 59, 474000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2023, 3, 24, 8, 28, 1, 796000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:108594546720:processing-job/pipelines-o0i36853z9qt-crude-palm-oil-price-ebaejxc2jb'}}},
 {'StepName': 'crude-palm-oil-prices-forecast-Preprocessing',
  'StartTime': datetime.datetime(2023, 3, 24, 7, 37, 6, 306000

In [24]:
train_response

{'StepName': 'crude-palm-oil-prices-forecast-Preprocessing',
 'StartTime': datetime.datetime(2023, 3, 24, 7, 37, 6, 306000, tzinfo=tzlocal()),
 'EndTime': datetime.datetime(2023, 3, 24, 7, 57, 58, 527000, tzinfo=tzlocal()),
 'StepStatus': 'Succeeded',
 'AttemptCount': 0,
 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:108594546720:processing-job/pipelines-o0i36853z9qt-crude-palm-oil-price-kmqquxaamk'}}}

In [26]:
processing_response = execution.list_steps()[0]
processing_arn = processing_response['Metadata']['ProcessingJob']['Arn'] # index -1은 가장 처음 실행 step
processing_job_name = processing_arn.split('/')[-1] # Processing job name만 추출
processing_response = sm_client.describe_processing_job(ProcessingJobName = processing_job_name)
processing_response

{'ProcessingInputs': [{'InputName': 'input-1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://crude-palm-oil-prices-forecast/trained-model/2023/03/23/1679675822.0/leaderboard',
    'LocalPath': '/opt/ml/processing/input/leaderboard',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}},
  {'InputName': 'code',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://crude-palm-oil-prices-forecast/src/model_validation.py',
    'LocalPath': '/opt/ml/processing/input/code',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingJobName': 'pipelines-o0i36853z9qt-crude-palm-oil-price-VCUzd43ncw',
 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 1,
   'InstanceType': 'ml.c5.xlarge',
   'VolumeSizeInGB': 30}},
 'StoppingCondition': {'MaxRuntimeInSeconds': 86400},
 'AppSpecification': {'ImageUri'

```
WaiterError: Waiter PipelineExecutionComplete failed: Max attempts exceeded
```


# 4. Future Works
참고사항: 앞으로 얼마나 예측을 할것인지에 대해서 split_date를 설정하여 앞으로 몇일을 예측할 수 있다.(현재 분기점은 '2022-10-31' 기준으로 되어있다.)

- Multi-model을 적용하는 multi pipeline
- CodeCommit을 통한 소스코드 관리
- sagemaker pipeline clarify 사용하여 data drift monitoring